# Lesson 6 : EPICS area detector

Start the instrument package as our routine initialization.

In [1]:
from instrument.collection import *

I Thu-09:45:00 - ############################################################ startup
I Thu-09:45:00 - logging started
I Thu-09:45:00 - logging level = 10
I Thu-09:45:00 - c:\Users\Pete\Documents\projects\use_bluesky\lessons\instrument\collection.py
I Thu-09:45:00 - c:\Users\Pete\Documents\projects\use_bluesky\lessons\instrument\mpl\notebook.py
Activating auto-logging. Current session state plus future input saved.
Filename       : c:\Users\Pete\Documents\projects\use_bluesky\lessons\.logs\ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
I Thu-09:45:02 - bluesky framework
I Thu-09:45:02 - c:\Users\Pete\Documents\projects\use_bluesky\lessons\instrument\framework\check_python.py
I Thu-09:45:02 - c:\Users\Pete\Documents\projects\use_bluesky\lessons\instrument\framework\check_bluesky.py
I Thu-09:45:04 - c:\Users\Pete\Documents\projects\use_bluesky\lessons\instrument\framework\initialize.py
I Thu-09:45:06 

The EPICS area detector support in ophyd has many features, some of them specific to the detector make and model.  Let's start with just a few features using the ADSimDetector.  We'll need to import support from ophyd by parts as they are needed.

As we add features from the area detector plugins or other capabilities, the configuration complexity will increase.  So, it is good to start with a simple case where we can control the camera and generate images.

In [2]:
from ophyd import Component
from ophyd.areadetector import ImagePlugin
from ophyd.areadetector import SimDetector
from ophyd.areadetector import SingleTrigger

In [3]:
# for jupyterlab ...
# https://stackoverflow.com/a/51932652
%matplotlib inline

In [4]:
_ad_prefix = "adsky:"

In [5]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")

In [6]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.wait_for_connection()

In [7]:
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25

adsimdet.stage_sigs

OrderedDict([('cam.acquire', 0),
             ('cam.image_mode', 1),
             ('cam.num_images', 1),
             ('cam.acquire_time', 0.1),
             ('cam.acquire_period', 0.25)])

In [8]:
RE(bp.count([adsimdet]))



Transient Scan ID: 1     Time: 2020-05-14 09:45:07
Persistent Unique Scan ID: '29f41ea9-57ee-492a-93fe-9fa76e2d2bed'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 09:45:08.1 |
+-----------+------------+
generator count ['29f41ea9'] (scan num: 1)


('29f41ea9-57ee-492a-93fe-9fa76e2d2bed',)

Get the detector image and display it.  In the initial instrument setup, MatPlotLib was initialized for our display.  We need a copy of the plotting object that was created, `plt` to show the detector image.

In [9]:
# get the MatPlotLib tool
from instrument.mpl import plt

In [10]:
# NOTE:
#   These next cells show how one might get the image data directly from EPICS
#   This is not recommended practice with Bluesky so the code has been commented out.

# # The image comes from EPICS AD as a 1-D array, row by row.
# # We need to make it into a 2-D array, so first we need to
# # get the number of rows and columns from the image plugin.

# array_size = adsimdet.image.array_size.get()
# shape = (array_size.height, array_size.width)

# # Now, get the image and make it a 2-D array.
# im = adsimdet.image.array_data.get().reshape(shape)

# # Tell MatPlotLib to show the image.
# plt.imshow(im)

It's useful to make these steps into a Python function.

In [11]:
# def show_ad_image(det):
#     array_size = det.image.array_size.get()
#     shape = (array_size.height, array_size.width)
#     im = det.image.array_data.get().reshape(shape)
#     plt.imshow(im)

In [12]:
# plt.close()   # stop interaction with previous MPL window
# show_ad_image(adsimdet)

----

Save image(s) to HDF5 file.

In [13]:
from ophyd.areadetector import ADComponent
from ophyd.areadetector import EpicsSignalWithRBV
from ophyd.areadetector import HDF5Plugin
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite

class MyHDF5Plugin(HDF5Plugin, FileStoreHDF5IterativeWrite):
    create_directory_depth = Component(EpicsSignalWithRBV, suffix="CreateDirectory")
    array_callbacks = Component(EpicsSignalWithRBV, suffix="ArrayCallbacks")

    pool_max_buffers = None
    
    def get_frames_per_point(self):
        return self.num_capture.get()

    def stage(self):
        super().stage()
        res_kwargs = {'frame_per_point': self.get_frames_per_point()}
        # res_kwargs = {'frame_per_point': self.num_capture.get()}
        self._generate_resource(res_kwargs)

In [14]:
DATABROKER_ROOT_PATH = "/tmp/"

# note: AD path MUST, must, MUST have trailing "/"!!!
#  ...and... start with the same path defined in root (above)

# path as seen by detector IOC
WRITE_HDF5_FILE_PATH = "/tmp/simdet/%Y/%m/%d/"
#!!! NOTE !!! This filesystem is on the IOC (might be in a docker container)!

# path as seen by databroker
READ_HDF5_FILE_PATH = "/tmp/docker_ioc/iocadsky/tmp/simdet/%Y/%m/%d/"

In [15]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")
    hdf1 = ADComponent(
        MyHDF5Plugin,
        suffix='HDF1:', 
        root=DATABROKER_ROOT_PATH,
        write_path_template = WRITE_HDF5_FILE_PATH,
        read_path_template = READ_HDF5_FILE_PATH,
    )

In [16]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25
adsimdet.hdf1.stage_sigs["num_capture"] = 1

adsimdet.read_attrs.append("hdf1")
if adsimdet.hdf1.create_directory_depth.get() == 0:
    # probably not set, so let's set it now to some default
    adsimdet.hdf1.create_directory_depth.put(-5)

NOTE: EPICS AreaDetector file-saving plugins (such as 
the HDF plugin) must be primed before they can be 
used.  Priming must be done in these situations:

* the IOC has just started
* the image size has changed

If you do not prime the detector and its plugin chain, *ophyd* will report an *UnprimedPlugin* error like this if you try to trigger (acquire data from) the detector:

```
UnprimedPlugin: The plugin hdf1 on the area detector with name DETECTOR_NAME has not been primed.
```

To prime the HDF plugin, call its `warmup()` method.

In [17]:
enabled = adsimdet.hdf1.enable.get()
adsimdet.hdf1.warmup()
adsimdet.hdf1.enable.put(enabled)

In [18]:
adsimdet.hdf1.stage_sigs

OrderedDict([('enable', 1),
             ('auto_increment', 'Yes'),
             ('array_counter', 0),
             ('auto_save', 'Yes'),
             ('num_capture', 1),
             ('file_template', '%s%s_%6.6d.h5'),
             ('file_write_mode', 'Stream'),
             ('capture', 1),
             ('blocking_callbacks', 'Yes'),
             ('parent.cam.array_callbacks', 1)])

In [19]:
print(f"frames/point: {adsimdet.hdf1.get_frames_per_point()}")
RE(bp.count([adsimdet]))

frames/point: 1


Transient Scan ID: 2     Time: 2020-05-14 09:45:14
Persistent Unique Scan ID: 'fda1331e-d95f-4ea9-bb18-6fbb2e97c182'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 09:45:14.8 |
+-----------+------------+
generator count ['fda1331e'] (scan num: 2)


('fda1331e-d95f-4ea9-bb18-6fbb2e97c182',)

----

Now, use databroker to retrieve that data and show the image.

In [20]:
h = db[-1]

In [21]:
for entry in h.documents():
    key, doc = entry
    print(key, doc)
    print("-"*40)

start
Run Start
beamline_id     : APS lesson                              
detectors       : ['adsimdet']                            
hints           :
  dimensions      : [[['time'], 'primary']]                 
login_id        : Pete@sev                                
num_intervals   : 0                                       
num_points      : 1                                       
pid             : 3740                                    
plan_args       :
  detectors       : ["MySingleTriggerSimDetector(prefix='adsky:', name='adsimdet', read_attrs=['hdf1'], configuration_attrs=['cam', 'cam.acquire_period', 'cam.acquire_time', 'cam.image_mode', 'cam.manufacturer', 'cam.model', 'cam.num_exposures', 'cam.trigger_mode', 'hdf1'])"]
  num             : 1                                       
plan_name       : count                                   
plan_type       : generator                               
proposal_id     : testing                                 
scan_id         : 

In [22]:
plt.close()   # stop interaction with previous MPL window
# FIXME: EventModelError: Error instantiating handler class  with Resource document {'spec': 'AD_HDF5', 'root': '/tmp', 'resource_path': 'docker_ioc/iocadsky/tmp/simdet/2020/05/13/ddfb7e48-4a25-4bf7-a45e_000000.h5', 'resource_kwargs': {'frame_per_point': 1}, 'path_semantics': 'posix', 'uid': '0117f6d5-67ca-4083-ba3b-6b30d5dca5f8', 'run_start': '94a857fe-a26d-4ede-947b-51e2187a8894'}. Its 'root' field /tmp was *not* modified by root_map.
plt.imshow(h.xarray().adsimdet_image[0][0])

EventModelError: Error instantiating handler class <class 'area_detector_handlers.handlers.AreaDetectorHDF5Handler'> with Resource document {'spec': 'AD_HDF5', 'root': '\\tmp', 'resource_path': 'docker_ioc\\iocadsky\\tmp\\simdet\\2020\\05\\14\\5227499c-6456-4caa-a419_000000.h5', 'resource_kwargs': {'frame_per_point': 1}, 'path_semantics': 'posix', 'uid': '336ef7a9-f114-4de6-a9cf-2cef0abc0cb2', 'run_start': 'fda1331e-d95f-4ea9-bb18-6fbb2e97c182'}. Its 'root' field \tmp was *not* modified by root_map.